# Mesh generation for tutorial 02

In [ ]:
import dolfinx.io
import gmsh
import mpi4py

In [ ]:
import multiphenicsx.io
import multiphenicsx.mesh

### Geometrical parameters

In [ ]:
pre_step_length = 4.
after_step_length = 14.
pre_step_height = 3.
after_step_height = 5.
lcar = 1. / 5.

### Create mesh with gmsh

In [ ]:
gmsh.initialize()
gmsh.model.add("mesh")
p0 = gmsh.model.geo.addPoint(0.0, after_step_height - pre_step_height, 0.0, lcar)
p1 = gmsh.model.geo.addPoint(pre_step_length, after_step_height - pre_step_height, 0.0, lcar)
p2 = gmsh.model.geo.addPoint(pre_step_length, 0.0, 0.0, lcar)
p3 = gmsh.model.geo.addPoint(pre_step_length + after_step_length, 0.0, 0.0, lcar)
p4 = gmsh.model.geo.addPoint(pre_step_length + after_step_length, after_step_height, 0.0, lcar)
p5 = gmsh.model.geo.addPoint(0.0, after_step_height, 0.0, lcar)
l0 = gmsh.model.geo.addLine(p0, p1)
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p5)
l5 = gmsh.model.geo.addLine(p5, p0)
line_loop = gmsh.model.geo.addCurveLoop([l0, l1, l2, l3, l4, l5])
domain = gmsh.model.geo.addPlaneSurface([line_loop])
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [l5], 1)
gmsh.model.addPhysicalGroup(1, [l0, l1, l2, l4], 2)
gmsh.model.addPhysicalGroup(2, [domain], 0)
gmsh.model.mesh.generate(2)

### Convert to a dolfinx mesh

In [ ]:
mesh, subdomains, boundaries = multiphenicsx.mesh.gmsh_to_fenicsx(gmsh.model, gdim=2)
gmsh.finalize()

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_tags(boundaries)

### Save mesh and boundaries

In [ ]:
with dolfinx.io.XDMFFile(mpi4py.MPI.COMM_WORLD, "backward_facing_step.xdmf", "w") as file:
    file.write_mesh(mesh)
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    file.write_meshtags(boundaries)